<a href="https://colab.research.google.com/github/akhilsrinath/soccer-analytics/blob/main/Database_querying_and_populating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Database population and querying**

#### Using Pandas and SQLAlchemy to store and retrieve StatsBomb event data

In [1]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [2]:
from sqlalchemy import create_engine

We use SQLAlchemy's `create_engine` function to create a temporary database in memory.



In [3]:
base_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/"
comp_url = base_url + "matches/{}/{}.json"
match_url = base_url + "events/{}.json"

In [4]:
def parse_data(competition_id, season_id):
    matches = requests.get(url=comp_url.format(competition_id, season_id)).json()
    match_ids = [m['match_id'] for m in matches]
    events = []
    for match_id in tqdm_notebook(match_ids):
        for e in requests.get(url=match_url.format(match_id)).json():
            events.append(e)
        
    return pd.json_normalize(events, sep='_')

In [5]:
competition_id = 43
season_id = 3
df = parse_data(competition_id, season_id)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/64 [00:00<?, ?it/s]

In [6]:
location_columns = [x for x in df.columns.values if 'location' in x]
for col in location_columns:
    for i, dimension in enumerate(["x", "y"]):
        new_col = col.replace("location", dimension)
        df[new_col] = df.apply(lambda x: x[col][i] if type(x[col]) == list else None, axis=1)

In [7]:
df.head(10)

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,location,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_end_location,pass_body_part_id,pass_body_part_name,pass_type_id,pass_type_name,carry_end_location,pass_outcome_id,pass_outcome_name,ball_receipt_outcome_id,ball_receipt_outcome_name,...,substitution_replacement_id,substitution_replacement_name,pass_cut_back,foul_committed_type_id,foul_committed_type_name,foul_won_defensive,dribble_nutmeg,shot_aerial_won,pass_goal_assist,foul_committed_card_id,foul_committed_card_name,clearance_aerial_won,pass_deflected,block_deflection,ball_recovery_offensive,shot_deflected,foul_committed_offensive,pass_miscommunication,pass_through_ball,pass_technique_id,pass_technique_name,shot_redirect,bad_behaviour_card_id,bad_behaviour_card_name,miscontrol_aerial_won,block_save_block,shot_open_goal,foul_committed_penalty,foul_won_penalty,shot_follows_dribble,x,y,pass_end_x,pass_end_y,carry_end_x,carry_end_y,shot_end_x,shot_end_y,goalkeeper_end_x,goalkeeper_end_y
0,07d17f3a-9ecc-4083-84c5-86022360a422,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,774,Egypt,1,Regular Play,774,Egypt,451.0,"[{'player': {'id': 5265, 'name': 'Mohamed El S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d546e5cd-ff15-47f6-b581-521fffb9e420,2,1,00:00:00.000,0,0,1,0.000,35,Starting XI,774,Egypt,1,Regular Play,783,Uruguay,442.0,"[{'player': {'id': 5267, 'name': 'Néstor Ferna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b5e89bb7-9ced-4968-bc86-9dc7b94d3710,3,1,00:00:00.000,0,0,1,NaN,18,Half Start,774,Egypt,1,Regular Play,774,Egypt,NaN,NaN,[667461fe-a1a2-45ba-a60f-b4b6f0d7d5c1],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,667461fe-a1a2-45ba-a60f-b4b6f0d7d5c1,4,1,00:00:00.000,0,0,1,9.975,18,Half Start,774,Egypt,1,Regular Play,783,Uruguay,NaN,NaN,[b5e89bb7-9ced-4968-bc86-9dc7b94d3710],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,490a99e5-f721-49c2-ba3b-19a3dd7e0a5f,5,1,00:00:00.507,0,0,2,1.920,30,Pass,783,Uruguay,9,From Kick Off,783,Uruguay,NaN,NaN,[dd300c89-21e5-4163-8de8-d77946ca0156],"[61.0, 41.0]",4319.0,Edinson Roberto Cavani Gómez,24.0,Left Center Forward,5259.0,José María Giménez de Vargas,33.105892,2.704966,1.0,Ground Pass,"[31.0, 55.0]",38.0,Left Foot,65.0,Kick Off,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,41.0,31.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN
5,dd300c89-21e5-4163-8de8-d77946ca0156,6,1,00:00:02.427,0,2,2,NaN,42,Ball Receipt*,783,Uruguay,9,From Kick Off,783,Uruguay,NaN,NaN,[490a99e5-f721-49c2-ba3b-19a3dd7e0a5f],"[31.0, 55.0]",5259.0,José María Giménez de Vargas,3.0,Right Center Back,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,411448bb-bd55-40b6-b7b0-a080aa

In [8]:
df = df[[c for c in df.columns if c not in location_columns]]

In [9]:
df.head()

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,tactics_lineup,related_events,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,pass_body_part_id,pass_body_part_name,pass_type_id,pass_type_name,pass_outcome_id,pass_outcome_name,ball_receipt_outcome_id,ball_receipt_outcome_name,ball_recovery_recovery_failure,under_pressure,interception_outcome_id,...,substitution_replacement_id,substitution_replacement_name,pass_cut_back,foul_committed_type_id,foul_committed_type_name,foul_won_defensive,dribble_nutmeg,shot_aerial_won,pass_goal_assist,foul_committed_card_id,foul_committed_card_name,clearance_aerial_won,pass_deflected,block_deflection,ball_recovery_offensive,shot_deflected,foul_committed_offensive,pass_miscommunication,pass_through_ball,pass_technique_id,pass_technique_name,shot_redirect,bad_behaviour_card_id,bad_behaviour_card_name,miscontrol_aerial_won,block_save_block,shot_open_goal,foul_committed_penalty,foul_won_penalty,shot_follows_dribble,x,y,pass_end_x,pass_end_y,carry_end_x,carry_end_y,shot_end_x,shot_end_y,goalkeeper_end_x,goalkeeper_end_y
0,07d17f3a-9ecc-4083-84c5-86022360a422,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,774,Egypt,1,Regular Play,774,Egypt,451.0,"[{'player': {'id': 5265, 'name': 'Mohamed El S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,d546e5cd-ff15-47f6-b581-521fffb9e420,2,1,00:00:00.000,0,0,1,0.000,35,Starting XI,774,Egypt,1,Regular Play,783,Uruguay,442.0,"[{'player': {'id': 5267, 'name': 'Néstor Ferna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b5e89bb7-9ced-4968-bc86-9dc7b94d3710,3,1,00:00:00.000,0,0,1,NaN,18,Half Start,774,Egypt,1,Regular Play,774,Egypt,NaN,NaN,[667461fe-a1a2-45ba-a60f-b4b6f0d7d5c1],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,667461fe-a1a2-45ba-a60f-b4b6f0d7d5c1,4,1,00:00:00.000,0,0,1,9.975,18,Half Start,774,Egypt,1,Regular Play,783,Uruguay,NaN,NaN,[b5e89bb7-9ced-4968-bc86-9dc7b94d3710],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,490a99e5-f721-49c2-ba3b-19a3dd7e0a5f,5,1,00:00:00.507,0,0,2,1.920,30,Pass,783,Uruguay,9,From Kick Off,783,Uruguay,NaN,NaN,[dd300c89-21e5-4163-8de8-d77946ca0156],4319.0,Edinson Roberto Cavani Gómez,24.0,Left Center Forward,5259.0,José María Giménez de Vargas,33.105892,2.704966,1.0,Ground Pass,38.0,Left Foot,65.0,Kick Off,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,41.0,31.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
columns_to_remove = ['tactics_lineup', 'related_events', 'shot_freeze_frame']
df = df[[c for c in df.columns if c not in columns_to_remove]]

In [11]:
engine = create_engine('sqlite://')

In [12]:
df.to_sql('events', engine)

In [13]:
top_passers = """
select player_name, count(*) as passes
from events
where 1=1
and type_name = "Pass"
group by player_id
order by count(*) desc
"""

pd.read_sql(top_passers, engine).head(10)

,player_name,passes
0,Luka Modrić,527
1,Sergio Ramos García,496
2,John Stones,479
3,Francisco Román Alarcón Suárez,475
4,Toby Alderweireld,446
5,Harry Maguire,434
6,Ivan Rakitić,426
7,Jordi Alba Ramos,414
8,Kieran Trippier,393
9,Kyle Walker,385
